In [4]:
#@title Initialize External Code
import time
import os
import subprocess
import shutil

# Store the current working directory
current_path = os.getcwd()

# Clear the /content/ directory

shutil.rmtree('\\content\\')
os.makedirs('\\content\\', exist_ok=True)

# Change the current working directory back to the original path
os.chdir(current_path)

start_time = time.time()

!nvidia-smi


Sun Aug 13 18:52:22 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.03                 Driver Version: 532.03       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti    WDDM | 00000000:07:00.0  On |                  N/A |
| 55%   73C    P0              186W / 350W|   1485MiB / 12288MiB |     32%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!git clone https://github.com/kalomaze/externalcolabcode.git /content/Retrieval-based-Voice-Conversion-WebUI/utils

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for utils Download: {elapsed_time} seconds")

Time taken for utils Download: 3.5484297275543213 seconds


Cloning into '/content/Retrieval-based-Voice-Conversion-WebUI/utils'...


In [8]:
#@title Install Dependencies and Clone Github Repository

import threading

start_time = time.time()

os.chdir("\\content\\Retrieval-based-Voice-Conversion-WebUI\\")
from utils.dependency import *
from utils.clone_alt import *
os.chdir("..")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for imports: {elapsed_time} seconds")

#@markdown This will forcefully update dependencies even after the initial install seemed to have functioned.
ForceUpdateDependencies = False #@param{type:"boolean"}
#@markdown This will force temporary storage to be used, so it will download dependencies every time instead of on Drive. Not needed, unless you really want that 160mb storage. (Turned on by default for non-training colab to boost the initial launch speed)
ForceTemporaryStorage = False #@param{type:"boolean"}

# Setup environment
print("Attempting to setup environment dependencies...")
print("\n----------------------------------------")

start_time_setup = time.time()
setup_environment(ForceUpdateDependencies, ForceTemporaryStorage)

# Apparently fastapi is getting errors as of writin according to #help-rvc
!pip install fastapi==0.88.0

end_time_setup = time.time()
elapsed_time_setup = end_time_setup - start_time_setup
print(f"Time taken for setup environment: {elapsed_time_setup} seconds")

print("----------------------------------------\n")
print("Attempting to clone necessary files...")
print("\n----------------------------------------")


Time taken for imports: 0.0 seconds
Attempting to setup environment dependencies...

----------------------------------------


ModuleNotFoundError: No module named 'google'

In [11]:

start_time_clone = time.time()
clone_repository(True)
!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -P /content/Retrieval-based-Voice-Conversion-WebUI/

end_time_clone = time.time()
elapsed_time_clone = end_time_clone - start_time_clone
print(f"Time taken for clone repository: {elapsed_time_clone} seconds")

print("----------------------------------------\n")
print("Cell completed.")

#total_time = elapsed_time + elapsed_time_setup + elapsed_time_clone
#print(f"Total time taken: {total_time} seconds")

!pip install -q stftpitchshift==1.5.1

Changing dir to /content/


Time taken for clone repository: 36.677645444869995 seconds
----------------------------------------

Cell completed.


--2023-08-13 19:13:46--  https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt
Resolving huggingface.co (huggingface.co)... 13.225.131.35, 13.225.131.93, 13.225.131.94, ...
Connecting to huggingface.co (huggingface.co)|13.225.131.35|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/ef/97/ef977053f017cde1fc0f89ac7ef0b33172a3d8fb8a840bb24e78b1c0f35f1e72/a5ed4719f59085d1affc5d81354c70828c740584f2d24e782523345a6a278962?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27rmvpe.pt%3B+filename%3D%22rmvpe.pt%22%3B&Expires=1692176285&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MjE3NjI4NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9lZi85Ny9lZjk3NzA1M2YwMTdjZGUxZmMwZjg5YWM3ZWYwYjMzMTcyYTNkOGZiOGE4NDBiYjI0ZTc4YjFjMGYzNWYxZTcyL2E1ZWQ0NzE5ZjU5MDg1ZDFhZmZjNWQ4MTM1NGM3MDgyOGM3NDA1ODRmMmQyNGU3ODI1MjMzNDVhNmEyNzg5NjI%7EcmVzcG9uc2UtY29ud

In [13]:
#@markdown ##Click this to load a DATASET instead.
DATASET = "PaulTrimmed.zip"  #@param {type:"string"}
#@markdown This will look for that zip inside your 'dataset' folder in google drive. (Fixed)

import os
import shutil
from glob import glob
import concurrent.futures
import subprocess

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._") or not filename.endswith(('.wav', '.flac', '.mp3', '.ogg', '.m4a')):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

def convert_file(source_file, output_filename_converted):
    # Check if the input file is 16-bit
    probe_cmd = f'ffprobe -v error -select_streams a:0 -show_entries stream=sample_fmt -of default=noprint_wrappers=1:nokey=1 "{source_file}"'
    sample_format = subprocess.run(probe_cmd, shell=True, text=True, capture_output=True).stdout.strip()

    # Use appropriate ffmpeg command based on sample format
    if sample_format == 's16':
        # Export as 16-bit WAV
        cmd = f'ffmpeg -i "{source_file}" -c:a pcm_s16le "{output_filename_converted}"'
    else:
        # Export as 32-bit float WAV (default behavior)
        cmd = f'ffmpeg -i "{source_file}" -c:a pcm_f32le "{output_filename_converted}"'

    process = subprocess.run(cmd, shell=True)
    if process.returncode != 0:
        print(f'Failed to convert {source_file}. The file may be corrupt.')
    else:
        os.remove(source_file)

def convert_audio_files(source_dir, dest_dir):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for root, _, files in os.walk(source_dir):
            for filename in files:
                file_ext = os.path.splitext(filename)[1].lower()
                if file_ext in ['.wav', '.flac', '.mp3', '.ogg', '.m4a']:
                    source_file = os.path.join(root, filename)
                    output_filename = os.path.join(dest_dir, filename)
                    output_filename_converted = os.path.splitext(output_filename)[0] + '_converted.wav'
                    executor.submit(convert_file, source_file, output_filename_converted)

dataset_path = '/content/drive/MyDrive/dataset/' + DATASET
final_directory = '/content/dataset'
temp_directory = '/content/temp_dataset'

try:
    if os.path.exists(final_directory):
        shutil.rmtree(final_directory)
        print("Dataset folder already found. Wiping clean for import operation...")
except Exception as e:
    print(f"Error in removing the final directory: {e}")

try:
    if not os.path.exists(dataset_path):
        raise Exception(f'There is no {DATASET} in {os.path.dirname(dataset_path)}')
except Exception as e:
    print(f"Error in verifying dataset: {e}")

try:
    os.makedirs(final_directory, exist_ok=True)
    os.makedirs(temp_directory, exist_ok=True)
except Exception as e:
    print(f"Error in creating directories: {e}")

try:
    # Unzip data into a temporary directory
    !unzip -d {temp_directory} -B {dataset_path}
except Exception as e:
    print(f"Error in unzipping data: {e}")

try:
    # Sanitize temporary directory
    sanitize_directory(temp_directory)
except Exception as e:
    print(f"Error in sanitizing directory: {e}")

try:
    # Convert audio files and move them to the final directory
    convert_audio_files(temp_directory, final_directory)
except Exception as e:
    print(f"Error in converting audio files: {e}")

try:
    # Clean up temp directory
    shutil.rmtree(temp_directory)
except Exception as e:
    print(f"Error in removing temp directory: {e}")

try:
    # Rename files if needed
    !rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' {final_directory}/*.*~*
except Exception as e:
    print(f"Error in renaming files: {e}")

print('Dataset imported. You can now copy the path of the dataset folder to the training path.')

Dataset folder already found. Wiping clean for import operation...
Error in verifying dataset: There is no PaulTrimmed.zip in /content/drive/MyDrive/dataset


'unzip'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


Dataset imported. You can now copy the path of the dataset folder to the training path.


명령 구문이 올바르지 않습니다.


In [ ]:
import os
import threading
import time
from utils import backups

#@markdown #Click here to run the GUI

#@markdown Keep this option enabled to use the simplified, easy interface.
easy_gui = False #@param{type:"boolean"}

#@markdown Keep this option enabled to use automatic backups feature.
auto_backups = True #@param{type:"boolean"}

#@markdown On this colab, the TensorBoard is automatically loaded. This lets you track advanced training statistics (not relevant for using existing models).
#@markdown <br> The important one being, loss/g/total. To avoid overtraining, watch the value and wait till it starts to rise again. <br>
#@markdown Overtraining is much more likely to happen on v2 weights. This is not necessarily a bad thing, as it means models will be finished sooner. <br>
#@markdown <div style="display:flex; justify-content:center">
#@markdown     <div style="margin-right:10px">
#@markdown         <img src="https://media.discordapp.net/attachments/1089076875999072302/1113732090391969792/image.png" width="700">
#@markdown     </div>
#@markdown </div>

#@markdown Follow the whole training guide here: <br> https://docs.google.com/document/d/13ebnzmeEBc6uzYCMt-QVFQk-whVrK4zw8k7_Lw3Bv_A/edit?usp=sharing

LOGS_FOLDER = '/content/Retrieval-based-Voice-Conversion-WebUI/logs'
if not os.path.exists(LOGS_FOLDER):
    os.makedirs(LOGS_FOLDER)

WEIGHTS_FOLDER = '/content/Retrieval-based-Voice-Conversion-WebUI/weights'
if not os.path.exists(WEIGHTS_FOLDER):
    os.makedirs(WEIGHTS_FOLDER)

def extract_wav2lip_tar_files():
    !wget -P /content/ https://github.com/777gt/EVC/raw/main/wav2lip-HD.tar.gz
    !wget -P /content/ https://github.com/777gt/EVC/raw/main/wav2lip-cache.tar.gz

    with tarfile.open('/content/wav2lip-cache.tar.gz', 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join('/', member.name)
            try:
                tar.extract(member, '/')
            except:
                pass
    with tarfile.open('/content/wav2lip-HD.tar.gz') as tar:
        tar.extractall('/content')

def start_web_server():
    %cd /content/Retrieval-based-Voice-Conversion-WebUI
    %load_ext tensorboard
    %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/audios
    if easy_gui:
        !pip install -q gTTS
        !pip install -q elevenlabs
        !wget https://github.com/777gt/-EVC-/raw/main/audios/someguy.mp3 -P /content/Retrieval-based-Voice-Conversion-WebUI/audios/
        !wget https://github.com/777gt/-EVC-/raw/main/audios/somegirl.mp3 -P /content/Retrieval-based-Voice-Conversion-WebUI/audios/
        extract_wav2lip_tar_files()
        !python3 EasierGUI.py --colab --pycmd python3
    else:
        !python3 infer-web.py --colab --pycmd python3

# Import the Google Drive backup before starting the server only if auto_backups is True
if auto_backups:
    backups.import_google_drive_backup()


# Start the web server in a separate thread
web_server_thread = threading.Thread(target=start_web_server)
web_server_thread.start()

# Run the backup loop in the main thread only if auto_backups is True
if auto_backups:
    backups.backup_files()
else:
    while True:
        time.sleep(10) # sleep for 10 seconds



In [ ]:
#@markdown # Save finished model(s) as zip files (to Drive)
#@markdown If you have completed training a model and want to save it
#@markdown for future use, without any extra files for training, you can run this cell.
#@markdown <br>Alternatively, you can manually create it by getting the added_ .index file (the added one not the 'trained' one) from `/RVC_Backup/` on your Drive and zipping it along with the .pth in `/RVC_Backup/weights/`

import os
import shutil
import zipfile
import re  # Import the regex module

logs_dir = '/content/Retrieval-based-Voice-Conversion-WebUI/logs/'
weights_dir = '/content/Retrieval-based-Voice-Conversion-WebUI/weights/'
output_dir = '/content/drive/MyDrive/RVC_Backup/Finished/'
zip_output_dir = '/content/finalzips_forcopying/'
directories = ['/content/finalsavetemp', '/content/finalzips_forcopying']

# Delete directories if they exist
for directory in directories:
    if os.path.exists(directory):
        shutil.rmtree(directory)

# Create the output directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(zip_output_dir, exist_ok=True)

# Find .pth files in weights directory
pth_files = [file for file in os.listdir(weights_dir) if file.endswith('.pth')]

skipped_files = set()  # Keep track of base names of skipped files

for pth_file in pth_files:
    match = re.search(r'(.*)_s\d+.pth$', pth_file)
    if match:  # Skip if file ends with _s followed by a number
        base_name = match.group(1)  # Get the base file name (before the `_s<number>` suffix)
        if base_name not in skipped_files:
            print(f'Skipping autosave epoch files for {base_name}.')
            skipped_files.add(base_name)
        continue

    print(f'Processing file: {pth_file}')
    folder = os.path.splitext(pth_file)[0]

    finalsavetemp_dir = '/content/finalsavetemp/'
    os.makedirs(finalsavetemp_dir, exist_ok=True)
    shutil.copy2(os.path.join(weights_dir, pth_file), finalsavetemp_dir)

    if os.path.isdir(os.path.join(logs_dir, folder)):
        index_files = [
            file for file in os.listdir(os.path.join(logs_dir, folder))
            if file.startswith('added') and file.endswith('.index')
        ]

        if index_files:
            latest_index_file = max(
                index_files,
                key=lambda x: os.path.getmtime(os.path.join(logs_dir, folder, x))
            )
            shutil.copy2(
                os.path.join(logs_dir, folder, latest_index_file),
                finalsavetemp_dir
            )
            print(f'Found matching .index file: {latest_index_file}')
        else:
            print("Model with no .index detected. Please regenerate using 'train feature index' if you are training it yourself")
    else:
        print("Model with no .index detected. Please regenerate using 'train feature index' if you are training it yourself")

    # Create zip and write files into it
    with zipfile.ZipFile(os.path.join(zip_output_dir, f'{folder}.zip'), 'w') as zipf:
        for root, dirs, files in os.walk(finalsavetemp_dir):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), finalsavetemp_dir))

    print(f'Created zip file for {folder}.')

    # Remove 'finalsavetemp'
    shutil.rmtree(finalsavetemp_dir)

    # Copy the finished zip file to the output directory
    shutil.copy2(os.path.join(zip_output_dir, f'{folder}.zip'), output_dir)
    print(f'Copied {folder}.zip to output directory: {output_dir}')

print('Backup process completed.')
print('Finished zips copied to /content/drive/MyDrive/RVC_Backup/Finished.')

# Delete directories if they exist
for directory in directories:
    if os.path.exists(directory):
        shutil.rmtree(directory)

In [ ]:
#@markdown # Backup your model(s) training data from /logs/ as .zip files (to your Drive)
#@markdown This colab should already auto backup everything in `/weights/` or `/logs/` to `/RVC_Backup/` on your drive if you have free space; therefore, this isn't necessary.
#@markdown <br> If that somehow fails, you can use this cell to make each folder in /logs/ a zip file, one by one.
import os
import shutil

logs_dir = '/content/Retrieval-based-Voice-Conversion-WebUI/logs/'
output_dir = '/content/drive/MyDrive/RVC_Backup/ManualTrainingBackup'

# Get a list of all folders in the logs directory
folders = [folder for folder in os.listdir(logs_dir) if os.path.isdir(os.path.join(logs_dir, folder))]

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over each folder and zip it
for folder in folders:
    folder_path = os.path.join(logs_dir, folder)
    zip_file_path = os.path.join(output_dir, f'{folder}_TrainingData.zip')  # Change here

    # Check if the zip file already exists in the output directory
    if os.path.exists(zip_file_path):
        existing_size = os.path.getsize(zip_file_path)
        shutil.make_archive(folder_path, 'zip', folder_path)
        new_size = os.path.getsize(f'{folder_path}.zip')
        if existing_size == new_size:
            print(f'Skipped {folder} because a zip with the same name and size already exists in {output_dir}')
            continue

    # Zip the folder
    shutil.make_archive(folder_path, 'zip', folder_path)

    # Move the zip file to the output directory
    shutil.move(f'{folder_path}.zip', zip_file_path)

    print(f'Zipped {folder} and saved it as {zip_file_path}')


In [ ]:
#@markdown #Text To Speech GUI (OUTDATED, USE THE BUILT IN TTS OF EASYGUI)
from IPython.display import clear_output
if not "tts_installed" in locals():
  %cd /content/
  !pip uninstall -y cmake
  !wget https://github.com/Kitware/CMake/releases/download/v3.22.1/cmake-3.22.1-linux-x86_64.tar.gz
  !tar xf cmake-3.22.1-linux-x86_64.tar.gz
  !rm cmake-3.22.1-linux-x86_64.tar.gz
  !PATH=$PATH:/content/cmake-3.22.1-linux-x86_64/bin
  import os
  os.environ["PATH"] += ":/content/cmake-3.22.1-linux-x86_64/bin"
  !apt-get install espeak
  !git clone https://github.com/log1stics/voice-generator-webui
  %cd voice-generator-webui
  !chmod +x setup.sh
  !./setup.sh
  clear_output()
  tts_installed=True
%cd /content/Retrieval-based-Voice-Conversion-WebUI/weights/
!for file in *.pth; do folder="${file%.pth}"; mkdir -p "/content/voice-generator-webui/vc/models/$folder"; cp "$file" "/content/voice-generator-webui/vc/models/$folder"; mv "$folder" "/content/voice-generator-webui/vc/models"; done
%cd /content/voice-generator-webui
!python3 webui.py --colab

In [ ]:
import os
import os.path
import shutil
#@title Manual Zip loading from Google Drive

#@markdown Model Zip Location

MODELLOCATION = "/content/drive/MyDrive/b10.zip"  #@param {type:"string"}

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!mkdir -p /content/unzips/{MODELNAME}/
!unzip -d /content/unzips/{MODELNAME} -B {MODELLOCATION}

# Set your source and destination directories
source_dir = os.path.join("/content/unzips/", MODELNAME)
#@markdown If you use this, you'll have to move the files manually.

In [ ]:
#@markdown #Step 2. Download The Model
#@markdown Link the URL path to the model (Mega, Drive, etc.) and start the code

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib

def calculate_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# Initialize gspread
scope = ['https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/drive']

config_path = '/content/Retrieval-based-Voice-Conversion-WebUI/stats/peppy-generator-388800-07722f17a188.json'

if os.path.exists(config_path):
    # File exists, proceed with creation of creds and client
    creds = Credentials.from_service_account_file(config_path, scopes=scope)
    client = gspread.authorize(creds)
else:
    # File does not exist, print message and skip creation of creds and client
    print("Sheet credential file missing.")

# Open the Google Sheet (this will write any URLs so I can easily track popular models)
book = client.open("RealtimeRVCStats")
sheet = book.get_worksheet(3)  # get the fourth sheet

def update_sheet(url, filename, filesize, md5_hash, index_version):
    data = sheet.get_all_records()
    df = pd.DataFrame(data)

    if md5_hash in df['MD5 Hash'].values:
        idx = df[df['MD5 Hash'] == md5_hash].index[0]

        # Update download count
        df.loc[idx, 'Download Counter'] = int(df.loc[idx, 'Download Counter']) + 1
        sheet.update_cell(idx+2, df.columns.get_loc('Download Counter') + 1, int(df.loc[idx, 'Download Counter']))

        # Find the next available Alt URL field
        alt_url_cols = [col for col in df.columns if 'Alt URL' in col]
        alt_url_values = [df.loc[idx, col_name] for col_name in alt_url_cols]

        # Check if url is the same as the main URL or any of the Alt URLs
        if url not in alt_url_values and url != df.loc[idx, 'URL']:
            for col_name in alt_url_cols:
                if df.loc[idx, col_name] == '':
                    df.loc[idx, col_name] = url
                    sheet.update_cell(idx+2, df.columns.get_loc(col_name) + 1, url)
                    break
    else:
        # Prepare a new row as a dictionary
        new_row_dict = {'URL': url, 'Download Counter': 1, 'Filename': filename,
                        'Filesize (.pth)': filesize, 'MD5 Hash': md5_hash, 'RVC Version': index_version}

        alt_url_cols = [col for col in df.columns if 'Alt URL' in col]
        for col in alt_url_cols:
            new_row_dict[col] = ''  # Leave the Alt URL fields empty

        # Convert fields other than 'Download Counter' and 'Filesize (.pth)' to string
        new_row_dict = {key: str(value) if key not in ['Download Counter', 'Filesize (.pth)'] else value for key, value in new_row_dict.items()}

        # Append new row to sheet in the same order as existing columns
        ordered_row = [new_row_dict.get(col, '') for col in df.columns]
        sheet.append_row(ordered_row, value_input_option='RAW')

condition1 = False
condition2 = False
already_downloaded = False

# condition1 here is to check if the .index was imported. 2 is for if the .pth was.

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

url = 'https://huggingface.co/kalomaze/KanyeV2_Redux/resolve/main/KanyeV2_REDUX_40khz.zip'  #@param {type:"string"}
model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

#@markdown This option should only be ticked if you don't want your model listed on the public tracker.
private_model = False #@param{type:"boolean"}

if url != '':
    MODEL = ""  # Initialize MODEL variable
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/weights/  # Create the 'weights' directory

    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Resolved URL:", url)  # Print the resolved URL
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Starting download from MEGA....")
        m.download_url(url, '/content/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Updated URL:", url)  # Print the updated URL
            url = url.replace("blob", "resolve")
            print("Resolved URL:", url)  # Print the resolved URL

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("No .zip file found on the page.")
            # Handle the case when no .zip file is found
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Outdated .pth detected! This is not compatible with the RVC method. Find the RVC equivalent model!")
                    continue  # Continue searching for a valid file
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Check file size in bytes (100MB)
                    print("Skipping unusable training file:", file)
                    continue  # Continue searching for a valid file
                return file_name
    return None

MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("Found .pth file:", MODEL + ".pth")
else:
    print("Error: Could not find a valid .pth file within the extracted zip.")
    print("If there's an error above this talking about 'Access denied', try one of the Alt URLs in the Google Sheets for this model.")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    # Move model into logs folder
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Found index file:", file)
                condition1 = True
                logs_folder = os.path.join('/content/Retrieval-based-Voice-Conversion-WebUI/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Create the logs folder if it doesn't exist

                # Delete identical .index file if it exists
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file)  # Set index_path variable

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'/content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("You already downloaded this model. Re-importing anyways..")
                    already_downloaded = True
                shutil.move(file_path, destination_path)
                condition2 = True
                if already_downloaded is False and os.path.exists(config_path):
                    file_size = os.path.getsize(destination_path) # Get file size
                    md5_hash = calculate_md5(destination_path) # Calculate md5 hash
                    index_version = find_version_number(index_path)  # Get the index version

if condition1 is False:
    logs_folder = os.path.join('/content/Retrieval-based-Voice-Conversion-WebUI/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason

if condition2 and not condition1:
    print("Model partially imported! No .index file was found in the model download. The author may have forgotten to add the index file.")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition1 and condition2:
    print("Model successfully imported!")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition3:
    pass  # Do nothing when condition3 is true
else:
    print("URL cannot be left empty. If you don't want to download a model now, just skip this step.")

!rm -r /content/unzips/
!rm -r /content/zips/

In [ ]:
#@title Easy Dataset Maker (not necessary to split manually on RVC anymore I believe)
#@markdown Just upload the .wav acapellas you want into the **EasyDataset** folder (make sure it is good quality stuff, please)
#@markdown Do not use spaces, weird symbols, or anything like that.
if not "pydub_installed" in locals():
  !pip install pydub --quiet
  pydub_installed=True

from pydub import AudioSegment
import os, shutil, wave
from google.colab import files
auto_delete_original_acapella=True#@param {type:"boolean"}

save_to_drive=True#@param {type:"boolean"}
#@markdown Choose a name for your dataset (you will use this later in the GUI)
dataset_name='mydataset'#@param {type:"string"}
if not "installed_pydub" in locals():
  !pip install pydub --quiet

if not os.path.exists('/content/EasyDataset'):
  !mkdir -p /content/EasyDataset
  os.chdir('/content/EasyDataset')
  print("Upload some files.")
  uploaded = files.upload()
else:
  os.chdir('/content/EasyDataset')
  if not os.listdir():
    print("Upload some files.")
    uploaded = files.upload()

for filename in os.listdir():
    if filename.endswith(".wav"):
        sound = AudioSegment.from_wav(filename)
        sound = sound.set_channels(1)
        new_filename = filename
        sound.export('mono_'+new_filename, format="wav")
        os.remove(filename)

# define the length of each clip in seconds
clip_length = 10
# iterate over all .wav files in the current directory
for filename in os.listdir():
    if not filename.endswith('.wav'):
        continue
    # open the WAV file and get the sample rate
    wav_file = wave.open(filename, 'rb')
    sample_rate = wav_file.getframerate()
    # calculate the number of frames in each clip
    clip_frames = clip_length * sample_rate
    # iterate over each clip and write it to a new file
    for i in range(int(wav_file.getnframes() / clip_frames) + 1):
        clip_name = f"{filename.split('.')[0]}_{i+1}.wav"
        clip_path = 'split_'+clip_name
        clip_start = i * clip_frames
        clip_end = min((i+1) * clip_frames, wav_file.getnframes())

        # write the clip to a new file
        with wave.open(clip_path, 'wb') as clip_file:
            clip_file.setparams(wav_file.getparams())
            clip_file.writeframes(wav_file.readframes(clip_end - clip_start))
    # close the original file
    wav_file.close()
    os.remove(filename)
!mkdir -p /content/dataset/{dataset_name}
for everything in os.listdir('/content/EasyDataset'):
  shutil.move(everything, f'/content/dataset/{dataset_name}')
!ls -a /content/dataset/
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/dataset/*.*~*
os.chdir(f'/content/dataset')
print(f"Your dataset has been saved to: /content/dataset/{dataset_name}")
if auto_delete_original_acapella:
  shutil.rmtree('/content/EasyDataset')
  !mkdir -p /content/EasyDataset
if save_to_drive:
  !zip -r {dataset_name}.zip {dataset_name}
  shutil.move(f'/content/dataset/{dataset_name}.zip','/content/drive/MyDrive/dataset')
os.chdir('/content')

In [ ]:
initialised = True
from time import sleep
from google.colab import output
from google.colab import drive

import sys
import os
import shutil
import psutil
import glob

mount_to_drive = True
mount_path = '/content/drive/MyDrive'

ai = 'https://github.com/kae0-0/Colab-for-MDX_B'
ai_version = 'https://github.com/kae0-0/Colab-for-MDX_B/raw/main/v'
onnx_list = 'https://raw.githubusercontent.com/kae0-0/Colab-for-MDX_B/main/onnx_list'
#@title Initialize UVR MDX-Net Models
#@markdown The 'ForceUpdate' option will update the models by fully reinstalling.
ForceUpdate = True #@param {type:"boolean"}
class h:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()
def console(t):
    get_ipython().system(t)
def LinUzip(file): # unzip call linux, force replace
    console(f'unzip -o {file}')
#-------------------------------------------------------
def getONNX():
    console(f'wget {onnx_list} -O onnx_list')
    _onnx = open("onnx_list", "r")
    _onnx = _onnx.readlines()
    os.remove('onnx_list')
    for model in _onnx:
        _model = sanitize_filename(os.path.basename(model))
        console(f'wget {model}')
        LinUzip(_model)
        os.remove(_model)

def getDemucs(_path):
    #https://dl.fbaipublicfiles.com/demucs/v3.0/demucs_extra-3646af93.th
    root = "https://dl.fbaipublicfiles.com/demucs/v3.0/"
    model = {
        'demucs_extra': '3646af93'
    }
    for models in zip(model.keys(),model.values()):
        console(f'wget {root+models[0]+"-"+models[1]}.th -O {models[0]}.th')
    for _ in glob.glob('*.th'):
        if os.path.isfile(os.path.join(os.getcwd(),_path,_)):
            os.remove(os.path.join(os.getcwd(),_path,_))
        shutil.move(_,_path)

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def toPath(path='local'):
    if path == 'local':
        os.chdir('/content')
    elif path == 'gdrive':
        os.chdir(mount_path)

def update():
    with h():
        console(f'wget {ai_version} -O nver')
    f = open('nver', 'r')
    nver = f.read()
    f = open('v', 'r')
    cver = f.read()
    if nver != cver or ForceUpdate:
        print('New update found! {}'.format(nver))
        os.chdir('../')
        print('Updating ai...',end=' ')
        with h():
            console(f'git clone {ai} temp_MDX_Colab')
            console('cp -a temp_MDX_Colab/* MDX_Colab/')
            console('rm -rf temp_MDX_Colab')
        print('done')
        os.chdir('MDX_Colab')
        print('Refreshing models...', end=' ')
        with h():
            getDemucs('model/')
            getONNX()
        print('done')
        output.clear()
        os.remove('v')
        os.rename("nver",'v')
        #os.chdir(f'{os.path.join(mount_path,"MDX_Colab")}')
    else:
        os.remove('nver')
        print('Using latest version.')

def past_installation():
    return os.path.exists('MDX_Colab')

def LoadMDX():
    console(f'git clone {ai} MDX_Colab')

#-------------------------------------------------------
# install requirements
print('Installing dependencies will take 45 seconds...',end=' ')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    svmem = psutil.virtual_memory()
    gpu_runtime = False
    with h():
        console('pip3 install onnxruntime==1.14.1')
else:
    gpu_runtime = True
    with h():
        console('pip3 install onnxruntime-gpu==1.14.1')
with h():
    deps = [
            'pathvalidate',
            'youtube-dl',
            'django'
    ]
    for dep in deps:
        console('pip3 install {}'.format(dep))
# import modules
#console('pip3 install torch==1.13.1')
console('pip3 install soundfile==0.12.1')
console('pip3 install librosa==0.9.1')
from pathvalidate import sanitize_filename
print('done')
if not gpu_runtime:
    print(f'GPU runtime is disabled. You have {get_size(svmem.total)} RAM.\nProcessing will be incredibly slow. 😈')
elif gpu_info.find('Tesla T4') >= 0:
    print('You got a Tesla T4 GPU. (speeds are around  10-25 it/s)')
elif gpu_info.find('Tesla P4') >= 0:
    print('You got a Tesla P4 GPU. (speeds are around  8-22 it/s)')
elif gpu_info.find('Tesla K80') >= 0:
    print('You got a Tesla K80 GPU. (This is the common gpu, speeds are around 2-10 it/s)')
elif gpu_info.find('Tesla P100') >= 0:
    print('You got a Tesla P100 GPU. (This is the Second to the fastest gpu, speeds are around  15-42 it/s)')
else:
    if gpu_runtime:
        print('You got an unknown GPU. Please report the GPU you got!')
        !nvidia-smi

#console('pip3 install demucs')
#-------------------------------------------------------
# Scripting
mount(mount_to_drive)
toPath('gdrive' if mount_to_drive else 'local')
#check for MDX existence
if not past_installation():
    print('First time installation will take around 3-6 minutes.\nThis requires around 2-3 GB Free Gdrive space.\nPlease try not to interup installation process!!')
    print('Downloading AI...',end=' ')
    with h():
        LoadMDX()
    os.chdir('MDX_Colab')
    print('done')

    print('Downloading models...',end=' ')
    with h():
        getDemucs('model/')
        getONNX()
    if os.path.isfile('onnx_list'):
        os.remove('onnx_list')
    print('done')

else:
    os.chdir('MDX_Colab')
    update()

################
#outro
print('Success!')

In [ ]:
#@markdown ##Click this to import a ZIP of AUDIO FILES (for isolation.)
#@markdown *Or*, you can import your audio files into the /tracks/ folder in the MDX_Colab folder stored in your Google Drive, and it will import them automatically when initializing. <br><br>
#@markdown Link the URL path to the audio files (Mega, Drive, etc.) and start the code
url = 'INSERTURLHERE'  #@param {type:"string"}

import subprocess
import os
import shutil
from urllib.parse import urlparse, parse_qs
from google.colab import output
from google.colab import drive


mount_to_drive = True
mount_path = '/content/drive/MyDrive'

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def check_package_installed(package_name):
    command = f"pip show {package_name}"
    result = subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return result.returncode == 0

def install_package(package_name):
    command = f"pip install {package_name} --quiet"
    subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if not check_package_installed("mega.py"):
    install_package("mega.py")

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

audio_zip = urlparse(url).path.split('/')[-2] + '.zip'
audio_zip_path = '/content/zips/' + audio_zip

if url != '':
    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$audio_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        m.download_url(url, '/content/zips')
    else:
        !wget "$url" -O "$audio_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

# Copy the unzipped audio files to the /content/drive/MyDrive/MDX_Colab/tracks folder
!mkdir -p /content/drive/MyDrive/MDX_Colab/tracks
for filename in os.listdir("/content/unzips"):
    if filename.endswith((".wav", ".mp3")):
        audio_file = os.path.join("/content/unzips", filename)
        destination_file = os.path.join("/content/drive/MyDrive/MDX_Colab/tracks", filename)
        shutil.copy2(audio_file, destination_file)
        if os.path.exists(destination_file):
            print(f"Copy successful: {destination_file}")
        else:
            print(f"Copy failed: {audio_file}")

!rm -r /content/unzips/
!rm -r /content/zips/


In [ ]:
#@markdown ### Print a list of tracks
for i in glob.glob('tracks/*'):
    print(os.path.basename(i))